In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 13:54:22.982286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 13:54:23.083162: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 13:54:23.083176: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 13:54:23.104021: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 13:54:23.626476: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100


2022-11-10 13:54:28.156008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 13:54:28.156524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:54:28.156571: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:54:28.156612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 13:54:28.156650: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 392ms/step - loss: 3.6323 - auc: 0.2607
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.2097 - auc: 0.7907
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 2.1341 - auc: 0.8931
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 2.2553 - auc: 0.9074
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 2.3230 - auc: 0.9048
Epoch 6/100
1/1 [==============================] - 0s 23ms/step - loss: 2.2813 - auc: 0.9088
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 2.1647 - auc: 0.9196
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 2.0289 - auc: 0.9271
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8820 - auc: 0.9366
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6955 - auc: 0.9500
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5263 

1/1 [==============================] - 0s 28ms/step - loss: 0.1730 - auc: 0.9967
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1663 - auc: 0.9977
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1666 - auc: 0.9977
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1634 - auc: 0.9974
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1715 - auc: 0.9969
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1701 - auc: 0.9968
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1623 - auc: 0.9977
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1599 - auc: 0.9978
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1648 - auc: 0.9972
Epoch 97/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1589 - auc: 0.9975
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [3]:
name='remove_rf'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 2.0783 - auc: 0.6804
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 2.0246 - auc: 0.8378
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.9142 - auc: 0.8710
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7683 - auc: 0.8713
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.5425 - auc: 0.9000
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4470 - auc: 0.9054
Epoch 7/100
1/1 [==============================] - 0s 23ms/step - loss: 1.3957 - auc: 0.9031
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3191 - auc: 0.9201
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1963 - auc: 0.9545
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1120 - auc: 0.9701
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1896 - auc: 0.9948
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1819 - auc: 0.9956
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1761 - auc: 0.9968
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1829 - auc: 0.9954
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1823 - auc: 0.9954
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1839 - auc: 0.9947
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1806 - auc: 0.9962
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1815 - auc: 0.9955
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1845 - auc: 0.9955
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1800 - auc: 0.9950
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [4]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 5.4330 - auc: 0.2215
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 3.1289 - auc: 0.3142
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8174 - auc: 0.8145
Epoch 4/100
1/1 [==============================] - 0s 24ms/step - loss: 1.6547 - auc: 0.9235
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.8445 - auc: 0.9195
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.9950 - auc: 0.9152
Epoch 7/100
1/1 [==============================] - 0s 40ms/step - loss: 2.0762 - auc: 0.9040
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 2.1362 - auc: 0.8990
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 2.1228 - auc: 0.8957
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 2.0379 - auc: 0.9014
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2195 - auc: 0.9964
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2081 - auc: 0.9980
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2156 - auc: 0.9965
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2093 - auc: 0.9963
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2049 - auc: 0.9975
Epoch 93/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2002 - auc: 0.9978
Epoch 94/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2062 - auc: 0.9967
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2018 - auc: 0.9970
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2047 - auc: 0.9970
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2060 - auc: 0.9954
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [5]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 341ms/step - loss: 2.9170 - auc: 0.3282
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9010 - auc: 0.8019
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8836 - auc: 0.8841
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.9883 - auc: 0.8976
Epoch 5/100
1/1 [==============================] - 0s 37ms/step - loss: 1.9934 - auc: 0.9044
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9624 - auc: 0.9070
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8552 - auc: 0.9157
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6730 - auc: 0.9282
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5043 - auc: 0.9410
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3599 - auc: 0.9522
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.1769 - auc: 0.9971
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1766 - auc: 0.9971
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1755 - auc: 0.9968
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1822 - auc: 0.9959
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1774 - auc: 0.9967
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1810 - auc: 0.9953
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1746 - auc: 0.9968
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1742 - auc: 0.9967
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1721 - auc: 0.9966
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1748 - auc: 0.9965
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [6]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 347ms/step - loss: 2.6853 - auc: 0.3640
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.8499 - auc: 0.8314
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8685 - auc: 0.8875
Epoch 4/100
1/1 [==============================] - 0s 40ms/step - loss: 1.9334 - auc: 0.9035
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.9063 - auc: 0.9092
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 1.8293 - auc: 0.9175
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6736 - auc: 0.9297
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5305 - auc: 0.9386
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3648 - auc: 0.9536
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2291 - auc: 0.9691
Epoch 11/100
1/1 [==============================] - 0s 25ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1613 - auc: 0.9976
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1561 - auc: 0.9976
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1526 - auc: 0.9978
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1575 - auc: 0.9974
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1573 - auc: 0.9979
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1506 - auc: 0.9982
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1500 - auc: 0.9981
Epoch 95/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1514 - auc: 0.9981
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1562 - auc: 0.9975
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1544 - auc: 0.9980
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [7]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 344ms/step - loss: 2.7801 - auc: 0.3383
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 1.8962 - auc: 0.8143
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.9257 - auc: 0.8727
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 2.0074 - auc: 0.8859
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9910 - auc: 0.8927
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8978 - auc: 0.9046
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7212 - auc: 0.9231
Epoch 8/100
1/1 [==============================] - 0s 25ms/step - loss: 1.5886 - auc: 0.9274
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4269 - auc: 0.9394
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3131 - auc: 0.9490
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1824 - auc: 0.9965
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1832 - auc: 0.9964
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1886 - auc: 0.9953
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1804 - auc: 0.9961
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1814 - auc: 0.9967
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1812 - auc: 0.9967
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1812 - auc: 0.9966
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1869 - auc: 0.9957
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1738 - auc: 0.9972
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1739 - auc: 0.9975
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [8]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('5model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.941777
1      remove_rf  0.945010
2   remove_dmpnn  0.937237
3      remove_lr  0.934840
4    remove_fcnn  0.941214
5     remove_xgb  0.940248
